In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint as DataPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD as LogisticRegression
import sys
from sklearn.metrics import roc_curve
import numpy as np

In [5]:
TEST_DATA = "../data/preprocessed_train_5M.csv"
iteration = 100
def ParseData(row):
	data = [float(feature) for feature in row.split(",")]
	return DataPoint(data[0], data[1:20])

data = sc.textFile(TEST_DATA).map(ParseData)
train, test = data.randomSplit([0.6, 0.4], seed = 11L)
train.cache()
LR = LogisticRegression.train(train, iteration)
predictions = LR.predict(test.map(lambda x: x.features))
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)

In [6]:
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(test.count())
print testErr

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 205.0 failed 1 times, most recent failure: Lost task 8.0 in stage 205.0 (TID 7341, localhost): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 5657404 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1431)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1419)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1418)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:141

In [ ]:
LR.clearThreshold()
scores = LR.predict(test.map(lambda x: x.features))
labelsAndScores = test.map(lambda lp: lp.label).zip(scores)
labelsAndScores_list = labelsAndScores.collect()


In [3]:
labelsAndScores_list = np.array(labelsAndScores_list)

NameError: name 'labelsAndScores_list' is not defined

In [ ]:
print labelsAndScores_list.shape

In [ ]:
tpr, fpr = roc_curve(labelsAndScores_list[0,:], labelsAndScores_list[1,:])
plt.figure(1)
plt.plot(fpr, tpr)